In [75]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [155]:
def r2(y_true, y_pred):
    n = len(y_true)
    r2 = 1 - ((1/n * np.sum((y_true - y_pred)**2))/(1/n * np.sum((y_true - np.mean(y_true))**2)))
    return r2

__1.__  В коде из методички реализуйте один или несколько из критериев останова (количество листьев, количество используемых признаков, глубина дерева и т.д.).

Критерии остнова: 

- Ограничение максимальной глубины дерева.


- Ограничение максимального количества листьев.


- Ограничение минимального количества $n$ объектов в листе.


- Останов в случае, когда все объекты в листе относятся к одному классу.

In [18]:
# сгенерируем данные
classification_data, classification_labels = datasets.make_classification(n_features=2, n_informative=2, 
                                                                 n_classes=2, n_redundant=0,
                                                                 n_clusters_per_class=1, random_state=5)

train_data, test_data, train_labels, test_labels = train_test_split(classification_data, 
                                                                    classification_labels, 
                                                                    test_size=0.3,
                                                                    random_state=1)

print(classification_data.shape)

(100, 2)


In [3]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [4]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
            
        # найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction        


$$H(X) = \sum^{K}_{k=1}p_{k}(1-p_{k}) = 1 - \sum_{k=1}^K{p_k^2} ,$$

In [5]:
# Расчет критерия Джини

def gini(labels):
    labels = list(labels)

    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    gini = 1
    for label in classes:
        p = classes[label] / len(labels)
        gini -= p ** 2
        
    return gini

$$H(X_{m}) - \frac{|X_{l}|}{|X_{m}|}H(X_{l}) - \frac{|X_{r}|}{|X_{m}|}H(X_{r}),$$

In [6]:
# Расчет прироста

def gain(left_labels, right_labels, root_gini):

    # доля выборки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return root_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [7]:
# Разбиение датасета в узле

def split(data, labels, column_index, t):
    
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)
        
    true_data = data[left]
    false_data = data[right]
    
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [8]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_samples_leaf = 5

    root_gini = gini(labels)

    best_gain = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique(data[:, index])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
                continue
            
            current_gain = gain(true_labels, false_labels, root_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index

In [34]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, max_deep=5, one_class_stop=False):

    gain, t, index = find_best_split(data, labels)
    
    # Ограничение глубины дерева
    if max_deep == 0:
        return Leaf(data, labels)

    # Остановка, если все объекты в листе одного класса
    if one_class_stop:
        leaf = Leaf(data, labels)
        if len(set(leaf.labels)) == 1:
            return Leaf(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if gain == 0:
        return Leaf(data, labels)

    # Случай с ограниченной глубиной дерева
    
    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
    # max_deep-=1

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, max_deep-1)
    import time
    print(time.time(), true_branch)
    false_branch = build_tree(false_data, false_labels, max_deep-1)
    
    print(time.time(), false_branch)
    
    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [12]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [13]:
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [35]:
my_tree = build_tree(train_data, train_labels,
                    #  max_deep=1, 
                     one_class_stop=True
                     )

1629623100.5399032 <__main__.Leaf object at 0x7fd89f255d50>
1629623100.540945 <__main__.Leaf object at 0x7fd89f255e10>
1629623100.541391 <__main__.Node object at 0x7fd89f255c90>
1629623100.5429788 <__main__.Leaf object at 0x7fd89f255290>
1629623100.5434399 <__main__.Node object at 0x7fd89f255cd0>
1629623100.5452614 <__main__.Leaf object at 0x7fd89f255a90>


In [25]:
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index), '<=', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на отрицательном поддереве
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")
    
print_tree(my_tree)

Индекс 0 <= 0.16261402870113306
--> True:
  Индекс 1 <= -1.5208896621663803
  --> True:
    Индекс 0 <= -0.9478301462477035
    --> True:
      Прогноз: 0
    --> False:
      Прогноз: 1
  --> False:
    Прогноз: 0
--> False:
  Прогноз: 1


__2*.__ Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.

In [51]:
# сгенерируем набор данных
X, Y, coef = datasets.make_regression(n_samples=1000, n_features=2, n_informative=2, n_targets=1, 
                                      noise=5, coef=True, random_state=2)

X[:, 0] *= 10

train_data, test_data, train_values, test_values = train_test_split(X, 
                                                                    Y, 
                                                                    test_size=0.3,
                                                                    random_state=1)


In [52]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [62]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, values):
        self.data = data
        self.values = values
        self.prediction = self.predict()
        
    def predict(self):
        # найдем среднее арифметическое от всех значений в листе и вернем его    
        prediction = np.mean(self.values)
        return prediction        

In [63]:
def mse(array):
    mean = array.mean()
    return np.mean((array - mean)**2)

In [64]:
# Расчет прироста

def gain(left_values, right_values, root_mse):

    # доля выборки, ушедшая в левое поддерево
    p = float(left_values.shape[0]) / (left_values.shape[0] + right_values.shape[0])
    
    return root_mse - p * mse(left_values) - (1 - p) * mse(right_values)

In [65]:
# Разбиение датасета в узле

def split(data, values, column_index, t):
    
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)
        
    true_data = data[left]
    false_data = data[right]
    
    true_values = values[left]
    false_values = values[right]
        
    return true_data, false_data, true_values, false_values

In [66]:
# Нахождение наилучшего разбиения

def find_best_split(data, values):
    
    #  обозначим минимальное количество объектов в узле
    min_samples_leaf = 5

    root_mse = mse(values)

    best_gain = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique(data[:, index])
        
        for t in t_values:
            true_data, false_data, true_values, false_values = split(data, values, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
                continue
            
            current_gain = gain(true_values, false_values, root_mse)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index

In [67]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, values, max_deep=5):

    gain, t, index = find_best_split(data, values)
    
    # Ограничение глубины дерева
    if max_deep == 0:
        return Leaf(data, values)


    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if gain == 0:
        return Leaf(data, values)

    # Случай с ограниченной глубиной дерева
    
    true_data, false_data, true_values, false_values = split(data, values, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_values, max_deep-1)
    import time
    print(time.time(), true_branch)
    false_branch = build_tree(false_data, false_values, max_deep-1)
    
    print(time.time(), false_branch)
    
    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [68]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [69]:
def predict(data, tree):
    
    values = []
    for obj in data:
        prediction = classify_object(obj, tree)
        values.append(prediction)
    return values

In [70]:
my_tree = build_tree(train_data, train_values)

1629626495.6758003 <__main__.Leaf object at 0x7fd89f235e10>
1629626495.6818473 <__main__.Leaf object at 0x7fd89f235f10>
1629626495.6848898 <__main__.Leaf object at 0x7fd89f235d90>
1629626495.6856427 <__main__.Node object at 0x7fd89f235fd0>
1629626495.685717 <__main__.Node object at 0x7fd89f276850>
1629626495.7210908 <__main__.Leaf object at 0x7fd89f302cd0>
1629626495.7274182 <__main__.Leaf object at 0x7fd89f235dd0>
1629626495.7282414 <__main__.Node object at 0x7fd89f276b10>
1629626495.7427998 <__main__.Leaf object at 0x7fd89f276350>
1629626495.7518928 <__main__.Leaf object at 0x7fd89f276c90>
1629626495.752625 <__main__.Node object at 0x7fd89f276a90>
1629626495.752698 <__main__.Node object at 0x7fd89f276550>
1629626495.752761 <__main__.Node object at 0x7fd89f238110>
1629626495.7787561 <__main__.Leaf object at 0x7fd89f276790>
1629626495.7806108 <__main__.Leaf object at 0x7fd89f276c50>
1629626495.7807 <__main__.Node object at 0x7fd89f276210>
1629626495.7842243 <__main__.Leaf object at 0x7

In [71]:
train_answers = predict(train_data, my_tree)
answers = predict(test_data, my_tree)

In [73]:
r2(train_values, train_answers)

0.9465266944201165